# Canvas Participation Analyzer - Example Usage

This notebook demonstrates how to:
1. Analyze student participation in Canvas forums and messages
2. Apply different grading schemes  
3. Integrate with student roster
4. Export results for gradebook

**Configuration:** All settings are managed in `config.py` - update it with your Canvas details before running.

## Setup and Configuration

**Before running this notebook:**
1. Edit `config.py` with your Canvas settings:
   - Set your Canvas API token
   - Update the course ID
   - Configure professor names for filtering
   - Adjust other settings as needed

2. Or create a `.env` file with your Canvas API token:
   ```
   CANVAS_API_TOKEN=your_token_here
   ```

In [ ]:
import os
import sys
import pandas as pd
from dotenv import load_dotenv
from participation_analyzer import analyze_participation, export_participation_data
from grading_schemes import apply_grading_scheme, load_student_roster, preview_grading_schemes, export_graded_data
import config

# Load environment variables from .env file
load_dotenv()

print("🎯 Canvas Participation Analyzer - Jupyter Notebook")
print("=" * 60)

## Configuration Check

In [ ]:
print("🔧 Configuration Check")
print("=" * 30)

# Check configuration from config.py
if not config.CANVAS_API_TOKEN:
    print("❌ Error: Canvas API token not found!")
    print("Please set CANVAS_API_TOKEN in config.py or as environment variable")
else:
    print(f"✅ Canvas API token: Set (ends with {config.CANVAS_API_TOKEN[-4:]})")

print(f"📋 Course ID: {config.COURSE_ID}")
print(f"✅ Canvas URL: {config.CANVAS_BASE_URL}")
print(f"🎓 Professor filter: {config.PROFESSOR_NAMES}")
print(f"📅 Start date: {config.SEMESTER_START_DATE}")
print(f"✅ Default grading scheme: {config.DEFAULT_GRADING_SCHEME}")

# Check data files
roster_file = "data_files/lista.csv"
if os.path.exists(roster_file):
    print(f"✅ Student roster: Found ({roster_file})")
else:
    print(f"⚠️  Student roster: Not found ({roster_file})")
    print("   This is optional but recommended")

print("\n💡 To configure:")
print("  • Edit config.py with your Canvas settings")
print("  • Or create a .env file with CANVAS_API_TOKEN")
print("  • Update COURSE_ID in config.py for your course")

## Step 1: Analyze Participation

In [ ]:
print("Step 1: Analyzing participation...")

participation_data = analyze_participation(
    course_id=config.COURSE_ID,
    canvas_token=config.CANVAS_API_TOKEN,
    include_forums=config.INCLUDE_FORUMS,
    include_messages=config.INCLUDE_MESSAGES
)

if participation_data.empty:
    print("⚠️  No participation data found. Check your course ID and API token.")
else:
    print(f"✅ Found participation data for {len(participation_data)} students")
    print(f"📊 Total participations range: {participation_data['total_participations'].min()} - {participation_data['total_participations'].max()}")
    
    # Display sample data
    print("\n📋 Sample participation data:")
    display(participation_data.head())

## Step 2: Export Raw Participation Data

In [ ]:
print("Step 2: Exporting raw participation data...")

export_participation_data(participation_data, "raw_participation_data.csv")
print("✅ Raw data exported to: raw_participation_data.csv")

## Step 3: Preview Grading Schemes

In [ ]:
print("Step 3: Previewing grading schemes...")

if not participation_data.empty:
    max_participations = participation_data['total_participations'].max()
    preview_range = list(range(0, min(int(max_participations) + 2, 12)))
    
    print(f"\n📊 Grading schemes preview (based on actual data range 0-{int(max_participations)}):")
    preview_grading_schemes(preview_range)
    
    print("\n📊 Recommendations:")
    print("• Tiered: Best for encouraging participation with clear milestones")
    print("• Linear: Fair proportional grading")
    print("• Logarithmic: Very generous to low participation")
    print("• Square Root: Balanced approach")
    print("• Percentage: Simple percentage-based")

## Step 4: Load Student Roster (Optional)

In [ ]:
print("Step 4: Loading student roster...")

student_roster = load_student_roster("data_files/lista.csv")

if student_roster is not None:
    print(f"✅ Student roster loaded: {len(student_roster)} students")
    print("\n📋 Sample roster data:")
    display(student_roster.head())
else:
    print("⚠️  No student roster found. Analysis will continue without roster integration.")

## Step 5: Apply Grading Scheme

In [ ]:
print("Step 5: Applying grading scheme...")

if not participation_data.empty:
    graded_data = apply_grading_scheme(
        participation_data,
        scheme_name=config.DEFAULT_GRADING_SCHEME,
        student_roster=student_roster
    )
    
    print(f"✅ Grading applied using '{config.DEFAULT_GRADING_SCHEME}' scheme")
    print(f"📊 Grade range: {graded_data['grade'].min():.1f} - {graded_data['grade'].max():.1f}")
    
    # Display sample graded data
    print("\n📋 Sample graded data:")
    display(graded_data.head())
else:
    print("❌ No participation data to grade")
    graded_data = pd.DataFrame()

## Step 6: Export Final Grades

In [ ]:
print("Step 6: Exporting final grades...")

if not participation_data.empty and not graded_data.empty:
    export_graded_data(graded_data, "participation_grades_final.csv")
    print("✅ Final grades exported to: participation_grades_final.csv")
else:
    print("❌ No data to export")

## Analysis Summary

In [ ]:
if not participation_data.empty and not graded_data.empty:
    print("=" * 60)
    print("📊 ANALYSIS SUMMARY")
    print("=" * 60)
    
    total_students = len(graded_data)
    avg_grade = graded_data['grade'].mean()
    avg_participation = graded_data['total_participations'].mean()
    
    print(f"Total students: {total_students}")
    print(f"Average participation: {avg_participation:.1f}")
    print(f"Average grade: {avg_grade:.2f}")
    print(f"Grade range: {graded_data['grade'].min():.1f} - {graded_data['grade'].max():.1f}")
    
    # Show top participants
    print(f"\n🏆 Top 5 participants:")
    top_participants = graded_data.nlargest(5, 'total_participations')
    for i, (_, student) in enumerate(top_participants.iterrows(), 1):
        name = student.get('user_name', 'Unknown')
        participations = student['total_participations']
        grade = student['grade']
        print(f"  {i}. {name}: {participations} participations → {grade:.1f}")
    
    # Show activity levels
    print(f"\n📈 Activity level distribution:")
    if 'activity_level' in graded_data.columns:
        activity_counts = graded_data['activity_level'].value_counts()
        for level, count in activity_counts.items():
            percentage = (count / total_students) * 100
            print(f"  {level}: {count} students ({percentage:.1f}%)")
    
    # Show grade distribution
    print(f"\n📊 Grade distribution:")
    grade_counts = graded_data['grade'].value_counts().sort_index()
    for grade, count in grade_counts.items():
        percentage = (count / total_students) * 100
        print(f"  {grade:.1f}: {count} students ({percentage:.1f}%)")
    
    print("\n✅ Analysis complete!")
    print(f"📁 Files created:")
    print(f"  • raw_participation_data.csv")
    print(f"  • participation_grades_final.csv")
else:
    print("❌ No data to analyze")

## Additional Analysis (Optional)

In [ ]:
# Optional: Create visualizations if matplotlib is available
try:
    import matplotlib.pyplot as plt
    
    if not participation_data.empty and not graded_data.empty:
        # Participation distribution
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 2, 1)
        plt.hist(graded_data['total_participations'], bins=10, alpha=0.7, color='skyblue')
        plt.title('Participation Distribution')
        plt.xlabel('Total Participations')
        plt.ylabel('Number of Students')
        plt.grid(True, alpha=0.3)
        
        # Grade distribution
        plt.subplot(1, 2, 2)
        plt.hist(graded_data['grade'], bins=10, alpha=0.7, color='lightgreen')
        plt.title('Grade Distribution')
        plt.xlabel('Grade')
        plt.ylabel('Number of Students')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Participation vs Grade scatter plot
        plt.figure(figsize=(8, 6))
        plt.scatter(graded_data['total_participations'], graded_data['grade'], alpha=0.6)
        plt.title('Participation vs Grade')
        plt.xlabel('Total Participations')
        plt.ylabel('Grade')
        plt.grid(True, alpha=0.3)
        plt.show()
        
    else:
        print("📊 No data available for visualization")
        
except ImportError:
    print("📊 Matplotlib not available. Install with: pip install matplotlib")

## Grading Schemes Demo

In [ ]:
print("🎯 Grading Schemes Demonstration")
print("=" * 50)

# Sample participation data
sample_participations = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

print("Sample participation counts:", sample_participations)
print()

# Preview all schemes
preview_grading_schemes(sample_participations)

## Next Steps

After running this analysis, you can:

1. **Review the exported files:**
   - `raw_participation_data.csv` - Raw participation data
   - `participation_grades_final.csv` - Final grades ready for import

2. **Customize the analysis:**
   - Modify `config.py` for your Canvas instance
   - Try different grading schemes
   - Adjust professor filtering
   - Set different date ranges

3. **Import to gradebook:**
   - Use the final CSV file to import grades to your LMS
   - Match student names/IDs with your roster

4. **Further analysis:**
   - Analyze participation patterns over time
   - Compare forum vs message participation
   - Track engagement trends